<a href="https://colab.research.google.com/github/Token-Creative-Services/isbnDB-API/blob/main/wordPressResources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WordPress Book Resources, Functions, Classes

In [ ]:
#A Class for a WordPress book. We would GET this data, and then pass it WordPress
#Should match ERD in --> https://lucid.app/lucidchart/5a8e7267-5339-4c01-a2cc-60b1075ce3cf/edit
class wordPressBook():
  """A class for all the details of a book that exists inside WordPress"""
  #Attributes for standard WordPress fields
  title = "";
  content = "";
  status = "";
  excerpt = "";
  featured_media = ""; #media ID, not URL
  #Attributes for ACF MANDATORY fields
  #bookTitle = ""; #same as basic wordPress title field
  book_author = "";
  publication_date = "";
  reference_link = "";
  isbn = "";
  genre = ""; #Currently only can send 1
  advanced_markup = ""; #can ommit for default No
  #Attributes for ACF OPTIONAL Fields
  book_format = "";
  amazon_paperback_url = "";
  amazon_hardcover_url = "";
  audiobook_url = "";
  kindle_url = "";
  publisher = "";
  offer_price = "";
  offer_price_currency = "";
  offer_eligible_country = "";
  offer_availability_status = "";
  book_edition_name = "";
  book_edition = "";
  platform_url_template = "";
  work_platforms = "";
  series = "";
  seriesid = "";
  pages = "";
  dewey_decimal = "";
  #Functional attributes
  createdBookId = ""; #used in our create book process, storing successfully created book so we can update the taxonomy

  #Methods/Functions
  def update_book_taxonomy_legacy(self):
    """Method that will take a book addded to WordPress, and update it to have the taxonomies it should. Returns True if it worked, None if there was a failure, and nothing if no matching tax was found"""
    #First we use /wp-json/acf/v3/genre to create an array of genreName/genreId. 
    getTaxonomyResponse = requests.get(WPBaseURL + '/wp-json/acf/v3/genre')
    if getTaxonomyResponse.status_code >= 500: #500 errors
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + WPBaseURL + '/wp-json/acf/v3/genre');
      print('[!] [{0}] Server Error'.format(getTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif getTaxonomyResponse.status_code == 404: #404 not found error
      print("update_book_taxonomy: Failed API call, route not found: " + WPBaseURL + '/wp-json/acf/v3/genre');
      print('[!] [{0}] 404 Not Found: [{1}]'.format(getTaxonomyResponse.status_code,api_url))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif getTaxonomyResponse.status_code == 401: #401 not auth
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + WPBaseURL + '/wp-json/acf/v3/genre');
      print('[!] [{0}] Authentication Failed'.format(getTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif getTaxonomyResponse.status_code >= 400: #400 bad requests
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + WPBaseURL + '/wp-json/acf/v3/genre');
      print('[!] [{0}] Bad Request, poorly formatted JSON'.format(getTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      print(getTaxonomyResponse.content )
      return None
    elif getTaxonomyResponse.status_code >= 300: #300 redirects
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + WPBaseURL + '/wp-json/acf/v3/genre');
      print('[!] [{0}] Unexpected redirect.'.format(getTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif getTaxonomyResponse.status_code == 200: #If getting the taxonomy succeeded
      getTaxonomyResponseJson = json.loads(getTaxonomyResponse.text);
      if loggingLevel == 1:
        print("update_book_taxonomy retrieved get taxonomy request: " + str(getTaxonomyResponseJson));
      #Iterate through that list and if the name matches a genre, return the ID
      for i in getTaxonomyResponseJson:
        try:
          currentgenre = str(i['acf']['name']);
          currentGenreId = str(i['id']);
          if loggingLevel == 1:
            print("update_book_taxonomoy: current genre in search is: " + str(currentgenre) + ". Current genre ID is: " + str(currentGenreId));
          if self.genre == currentgenre: #if and when we finally get a match of a genre inside WordPress...
            if loggingLevel == 1:
              print("update_book_taxonomy: Genre name matches! Iteration item is: " + str(i['acf']['name']));
              print("update_book_taxonomy API call being made to update book genre: " + WPBaseURL + '/wp-json/wp/v2/book/'+str(self.createdBookId) + '?genre='+currentGenreId)
            #Make the API call to update the book, applying one or more genreIds as a query param
            AddToTaxnomoyApiRequestURL = WPBaseURL + '/wp-json/wp/v2/book/'+str(self.createdBookId) + '?genre='+currentGenreId;
            addToTaxonomyResponse = requests.post(AddToTaxnomoyApiRequestURL, headers=wordPressHeaders) #POST to update taxonomy
            ##
            if addToTaxonomyResponse.status_code >= 500: #500 errors
              print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
              print('[!] [{0}] Server Error'.format(addToTaxonomyResponse.status_code))
              if loggingLevel == 1:
                print("update_book_taxonomy returning None on get taxony list");
              return None
            elif addToTaxonomyResponse.status_code == 404: #404 not found error
              print("update_book_taxonomy: Failed API call, route not found: " + AddToTaxnomoyApiRequestURL);
              print('[!] [{0}] 404 Not Found: [{1}]'.format(addToTaxonomyResponse.status_code,api_url))
              if loggingLevel == 1:
                print("update_book_taxonomy returning None on get taxony list");
              return None
            elif addToTaxonomyResponse.status_code == 401: #401 not auth
              print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
              print('[!] [{0}] Authentication Failed'.format(addToTaxonomyResponse.status_code))
              if loggingLevel == 1:
                print("update_book_taxonomy returning None on get taxony list");
              return None
            elif addToTaxonomyResponse.status_code >= 400: #400 bad requests
              print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
              print('[!] [{0}] Bad Request, poorly formatted JSON'.format(addToTaxonomyResponse.status_code))
              if loggingLevel == 1:
                print("update_book_taxonomy returning None on get taxony list");
              print(addToTaxonomyResponse.content )
              return None
            elif addToTaxonomyResponse.status_code >= 300: #300 redirects
              print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
              print('[!] [{0}] Unexpected redirect.'.format(addToTaxonomyResponse.status_code))
              if loggingLevel == 1:
                print("update_book_taxonomy returning None on get taxony list");
              return None
            elif addToTaxonomyResponse.status_code == 200: #If getting the taxonomy succeeded
              if loggingLevel == 1:
                print("update_book_taxonomy returning TRUE on update book taxonomy");
              return True;
        except:
          if loggingLevel == 1:
              print("update_book_taxonomy returning None in taxonomy crawl to find matching tax's");
          return None; 

  def update_book_taxonomy(self):
    """Method that will given a static list of IDs that we know. Working around what seems to be an issue/defect with genres not listing in the API"""
    #First we asses what all our current genres are as cited in /wp-admin/edit-tags.php?taxonomy=genre&post_type=book. COVERS THE DEFECT OF NOT GETTING ALL GENRES IN API CALL
    if self.genre == "Adventure Stories":
      currentGenreId = "4";
    elif self.genre == "Crime":
      currentGenreId = "8";
    elif self.genre == "Dystopia":
      currentGenreId = "3";
    elif self.genre == "Fantasy Fiction":
      currentGenreId = "5";
    elif self.genre == "Fiction":
      currentGenreId = "18";
    elif self.genre == "Historical Fiction":
      currentGenreId = "10";
    elif self.genre == "Humor":
      currentGenreId = "26";
    elif self.genre == "Love/Romance":
      currentGenreId = "7";
    elif self.genre == "Nonfiction":
      currentGenreId = "27";
    elif self.genre == "Science Fiction":
      currentGenreId = "2";
    elif self.genre == "Thriller":
      currentGenreId = "9";
    elif self.genre == "Young Adult Fiction":
      currentGenreId = "6";
    else: #if nothing we recognize list it under Uncategorized
      currentGenreId = "29";
  
    if loggingLevel == 1:
      print("update_book_taxonomy API call being made to update book genre: " + WPBaseURL + '/wp-json/wp/v2/book/'+str(self.createdBookId) + '?genre='+currentGenreId)
    #Make the API call to update the book, applying one or more genreIds as a query param
    AddToTaxnomoyApiRequestURL = WPBaseURL + '/wp-json/wp/v2/book/'+str(self.createdBookId) + '?genre='+currentGenreId;
    addToTaxonomyResponse = requests.post(AddToTaxnomoyApiRequestURL, headers=wordPressHeaders) #POST to update taxonomy
    if addToTaxonomyResponse.status_code >= 500: #500 errors
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
      print('[!] [{0}] Server Error'.format(addToTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif addToTaxonomyResponse.status_code == 404: #404 not found error
      print("update_book_taxonomy: Failed API call, route not found: " + AddToTaxnomoyApiRequestURL);
      print('[!] [{0}] 404 Not Found: [{1}]'.format(addToTaxonomyResponse.status_code,api_url))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif addToTaxonomyResponse.status_code == 401: #401 not auth
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
      print('[!] [{0}] Authentication Failed'.format(addToTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif addToTaxonomyResponse.status_code >= 400: #400 bad requests
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
      print('[!] [{0}] Bad Request, poorly formatted JSON'.format(addToTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      print(addToTaxonomyResponse.content )
      return None
    elif addToTaxonomyResponse.status_code >= 300: #300 redirects
      print("update_book_taxonomy: Failed API call to POST add book to wordpress: " + AddToTaxnomoyApiRequestURL);
      print('[!] [{0}] Unexpected redirect.'.format(addToTaxonomyResponse.status_code))
      if loggingLevel == 1:
        print("update_book_taxonomy returning None on get taxony list");
      return None
    elif addToTaxonomyResponse.status_code == 200: #If getting the taxonomy succeeded
      if loggingLevel == 1:
        print("update_book_taxonomy returning TRUE on update book taxonomy");
      return True;

  def upload_image_wordpress(self):
    """Upload an image to WordPress, accepting an image URL and ISBN#. Returns an int ID if the image was uploaded, or returns None if failure uploading image"""
    #First we download the image based on the current URL in the featured_media field
    if loggingLevel == 1:
        print("upload_image_wordpress attempting to retrieve image by URL");
    ImageUploadResponse = urllib.request.urlretrieve(self.featured_media, "{0}.jpg".format(self.isbn)); #get, save image locally
    imagePath = ImageUploadResponse[0]; #save the path of the locally saved image
    if imagePath == "":
      print("upload_image_wordpress: Failed to download image by URL from isbnDB URL: " + str(ImageUploadResponse));
      if loggingLevel == 1:
        print("upload_image_wordpress returning NONE, failed to download imge from URL");
      return None
    #Next we then take locally downloaded image and upload it to WordPress
    media = {'file': open(imagePath,'rb')} #Load as a binary file
    image = requests.post(WPBaseURL + '/wp-json/wp/v2/media/', headers=wordPressHeaders, files=media) #create a post request with the file
    if image == "":
      print("upload_image_wordpress: Failed to upload image to WordPress" + WPBaseURL + '/wp-json/wp/v2/media/')
      if loggingLevel == 1:
        print("upload_image_wordpress returning NONE as we failed to upload image to wordpress");
      return None
    link = json.loads(image.content.decode('utf-8'))['link'] # get the link to the image out of the response
    uploadedImageId =json.loads(image.content.decode('utf-8'))['id'] # get the post-id of the image out of the response (we will use it further down to change the wp-image parameters)
    if loggingLevel == 1:
      print('Your image is published on {0} with ID {1}'.format(link, uploadedImageId))
    #Second API call PUT to update the image with relevant meta info
    post = {'caption': self.title + ' book cover',
          'description': self.title + ' book cover'
          }
    if loggingLevel == 1:
        print("upload_image_wordpress making API call to update image: " + WPBaseURL + '/wp-json/wp/v2/media/'+str(uploadedImageId));
    r = requests.post(WPBaseURL + '/wp-json/wp/v2/media/'+str(uploadedImageId), headers=wordPressHeaders, json=post) # make now a standard post request this time with the json 'post' as content
    if r == "":
      print("upload_image_wordpress: Failed to update image metadata. API call failed: " + WPBaseURL + '/wp-json/wp/v2/media/'+str(uploadedImageId));
      if loggingLevel == 1:
        print("upload_image_wordpress returning NONE image upload API failed to wordPress");
      return None
    #delete the image from local storage once we are done with it
    try:
      os.remove(imagePath);
      if loggingLevel == 1:
        print("upload_image_wordpress image removed from local OS successfully");
    except:
      print("upload_image_wordpress: Failed to remove image from local OS. Could not find image by path: " + str(imagePath));
    if loggingLevel == 1:
        print("upload_image_wordpress returning success with the uploaded image's ID in WordPress");
    return str(uploadedImageId);
  
  def addBookToWordPress(self):
    """Method to add a book to WordPress. Returns None if failure was seen"""
    #First API call we download the image as a URL, and then set the ID feature image to ID of the image we just uploaded
    try:
      self.featured_media = self.upload_image_wordpress();
      if self.featured_media is None: #if for some reason the upload method returned a None (failure)
        print("addBookToWordPress: Image uploading failed. No featured_media ID provided for current book object" + str(self.featured_media));
      else:
        print("addBookToWordPress: Image uploaded successfully and applied to locla book object as: " + str(self.featured_media));
    except:
      print("addBookToWordPress: Image uploading failed. Exception found in upload_image_wordpress(). Adding book will continue...")
    
    #Secod API call POST to create a new book item
    addApiRequestURL = WPBaseURL + '/wp-json/wp/v2/book?content=' + self.content + '&title=' + self.title + '&status=' + self.status + '&excerpt=' + self.excerpt + '&featured_media=' + str(self.featured_media);     #Construct the URL to make the book
    if loggingLevel == 1:
        print("addBookToWordPress: making add book API call to WP...: " + str(addApiRequestURL));
        print("##############addBookToWordPress: title to note is " + self.title);
    addResponse = requests.post(addApiRequestURL, headers=wordPressHeaders)    #Make request
    if addResponse.status_code >= 500: #500 errors
      print("addBookToWordPress: Failed API call to POST add book to wordpress: " + addApiRequestURL);
      print('[!] [{0}] Server Error'.format(addResponse.status_code))
      if loggingLevel == 1:
        print("addBookToWordPress returning NONE failed to create new book");
      return None
    elif addResponse.status_code == 404: #404 not found error
      print("addBookToWordPress: Failed API call to POST add book to wordpress: " + addApiRequestURL);
      print('[!] [{0}] 404 Not Found: [{1}]'.format(addResponse.status_code,api_url))
      if loggingLevel == 1:
        print("addBookToWordPress returning NONE failed to create new book");
      return None
    elif addResponse.status_code == 401: #401 not auth
      print("addBookToWordPress: Failed API call to POST add book to wordpress: " + addApiRequestURL);
      print('[!] [{0}] Authentication Failed'.format(addResponse.status_code))
      if loggingLevel == 1:
        print("addBookToWordPress returning NONE failed to create new book");
      return None
    elif addResponse.status_code >= 400: #400 bad requests
      print("addBookToWordPress: Failed API call to POST add book to wordpress: " + addApiRequestURL);
      print('[!] [{0}] Bad Request, poorly formatted JSON'.format(addResponse.status_code))
      if loggingLevel == 1:
        print("addBookToWordPress returning NONE failed to create new book");
        print(addResponse.content)
      return None
    elif addResponse.status_code >= 300: #300 redirects
      print("addBookToWordPress: Failed API call to POST add book to wordpress: " + addApiRequestURL);
      print('[!] [{0}] Unexpected redirect.'.format(addResponse.status_code))
      if loggingLevel == 1:
        print("addBookToWordPress returning NONE failed to create new book");
      return None
    elif addResponse.status_code == 201: #If creating book worked, we continue...
      AddJson_response = json.loads(addResponse.text);
      self.createdBookId = str(AddJson_response['id']); #assign the book that was created
      if loggingLevel == 1:
        print("addBookToWordPress: Book created with an ID: " + str(self.createdBookId));
      
      #Third we add the taxonomy details for genre.
      try:
        updateToAddTaxonomy = self.update_book_taxonomy()
        if updateToAddTaxonomy is None: #if for some reason the upload method returned a None (failure)
          print("addBookToWordPress: API call to update taxonomy failed, returning None implying API failure")
        else: #If its not none it probably worked
          if loggingLevel == 1:
            print("addBookToWordPress: API call to update taxonomy succeeded: " + str(updateToAddTaxonomy));
      except:
        print("addBookToWordPress: API call to update taxonomy failed. Try/Catch failure seen")
        #no returns, it worked so we let code continue to next section for UPDATE
    else:
      print("addBookToWordPress: Failed API call to update taxonomy failed: " + addApiRequestURL);
      print('[?] Unexpected Error: [HTTP {0}]: Content: {1}'.format(addResponse.status_code, addResponse.content))
      if loggingLevel == 1:
        print("addBookToWordPress returning NONE failed in process of trying to update taxonomy");
      return None
    
    #Forth, we Take the Just Created Book, And Update It's ACF's
    try:
      updateACFAttempt = self.updateACFFields();
      if updateACFAttempt == True:
        if loggingLevel == 1:
          print("addBookToWordPress: API to update ACF fields for book successfull");
      elif updateACFAttempt == None:
        print("addBookToWordPress: API to update ACF failed" + updateACFAttempt);
    except:
      print("addBookToWordPress: API to update ACF failed" + updateACFAttempt);

  def updateACFFields(self):
    """Updates the ACF fields for a given book. Returns True if it worked, returns None if it failed """
    data = "{\"fields\":{" #Starting caps
    if self.isbn is not "":
      data = data + "\"isbn\":\"" + self.isbn + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting isbn:" + self.isbn);
    if self.title is not "":
      data = data + "\"title\":\"" + self.title + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting title:" + self.title);
    if self.book_author is not "":
      data = data + "\"book_author\":\"" + self.book_author + "\",";   #####################################
      if loggingLevel == 1:
        print("updateACFFields: Setting book_author:" + self.book_author);
    if self.publication_date is not "":
      data = data + "\"publication_date\":\"" + self.publication_date + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting publication_date:" + self.publication_date);
    if self.reference_link is not "":
      data = data + "\"reference_link\":\"" + self.reference_link + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting reference_link:" + self.reference_link);
    if self.genre is not "": #ACF genre, not core book genre
      data = data + "\"genre\":\"" + self.genre + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting genre:" + self.genre);
    if self.advanced_markup is not "":
      data = data + "\"advanced_markup\":\"" + self.advanced_markup + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting advanced_markup:" + self.advanced_markup);
    if self.book_format is not "":
      data = data + "\"book_format\":\"" + self.book_format + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting book_format:" + self.book_format);
    if self.amazon_paperback_url is not "":
      data = data + "\"amazon_paperback_url\":\"" + self.amazon_paperback_url + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting amazon_paperback_url:" + self.amazon_paperback_url);
    if self.amazon_hardcover_url is not "":
      data = data + "\"amazon_hardcover_url\":\"" + self.amazon_hardcover_url + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting amazon_hardcover_url:" + self.amazon_hardcover_url);
    if self.audiobook_url is not "":
      data = data + "\"audiobook_url\":\"" + self.audiobook_url + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting audiobook_url:" + self.audiobook_url);
    if self.kindle_url is not "":
      data = data + "\"kindle_url\":\"" + self.kindle_url + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting kindle_url:" + self.kindle_url);
    if self.publisher is not "":
      data = data + "\"publisher\":\"" + self.publisher + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting publisher:" + self.publisher);
    if self.offer_price is not "":
      data = data + "\"offer_price\":\"" + self.offer_price + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting offer_price:" + self.offer_price);
    if self.offer_price_currency is not "":
      data = data + "\"offer_price_currency\":\"" + self.offer_price_currency + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting offer_price_currency:" + self.offer_price_currency);
    if self.offer_eligible_country is not "":
      data = data + "\"offer_eligible_country\":\"" + self.offer_eligible_country + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting offer_eligible_country:" + self.offer_eligible_country);
    if self.offer_availability_status is not "":
      data = data + "\"offer_availability_status\":\"" + self.offer_availability_status + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting offer_availability_status:" + self.offer_availability_status);
    if self.book_edition_name is not "":
      data = data + "\"book_edition_name\":\"" + self.book_edition_name + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting book_edition_name:" + self.book_edition_name);
    if self.platform_url_template is not "":
      data = data + "\"platform_url_template\":\"" + self.platform_url_template + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting platform_url_template:" + self.platform_url_template);
    if self.work_platforms is not "":
      data = data + "\"work_platforms\":\"" + self.work_platforms + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting work_platforms:" + self.work_platforms);
    if self.series is not "":
      data = data + "\"series\":\"" + self.series + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting series:" + self.series);
    if self.seriesid is not "":
      data = data + "\"seriesid\":\"" + self.seriesid + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting seriesid:" + self.seriesid);
    if self.pages is not "":
      data = data + "\"pages\":\"" + self.pages;
      if loggingLevel == 1:
        print("updateACFFields: Setting pages:" + self.pages);
    if self.platform_url_template is not "":
      data = data + "\"platform_url_template\":\"" + self.platform_url_template + "\",";
      if loggingLevel == 1:
        print("updateACFFields: Setting platform_url_template:" + self.platform_url_template);
    if self.platform_url_template is not "":
      data = data + "\"platform_url_template\":\"" + self.platform_url_template;
      if loggingLevel == 1:
        print("updateACFFields: Setting platform_url_template:" + self.platform_url_template);
    data = data + "\"}}"; #ending cap
    if loggingLevel == 1:
      print("updateACFFields: JSON fully compiled: " + data);
    updateApiRequestURL = WPBaseURL + '/wp-json/acf/v3/book/'+ self.createdBookId;
    if loggingLevel == 1:
      print("updateACFFields: making API call to update book ACT fields: " + updateApiRequestURL);
    updateResponse = requests.post(updateApiRequestURL, headers=acfUpdateHeaders,data=data);
    if updateResponse.status_code >= 500: #500 errors
      print("updateACFFields: Failed API call to update ACF Fields: " + updateApiRequestURL);
      print('[!] [{0}] Server Error'.format(updateResponse.status_code))
      if loggingLevel == 1:
        print("updateACFFields: returning None API to update ACF fields failed with 500 error");
      return None
    elif updateResponse.status_code == 404: #404 not found error
      print("updateACFFields: Failed API call to update ACF Fields: " + updateApiRequestURL);
      print('[!] [{0}] 404 Not Found: [{1}]'.format(updateResponse.status_code,api_url))
      if loggingLevel == 1:
        print("updateACFFields: returning None API to update ACF fields failed with 404 error");
      return None
    elif updateResponse.status_code == 401: #401 not auth
      print("updateACFFields: Failed API call to update ACF Fields: " + updateApiRequestURL);
      print('[!] [{0}] Authentication Failed'.format(updateResponse.status_code))
      if loggingLevel == 1:
        print("updateACFFields: returning None API to update ACF fields failed with 401 error");
      return None
    elif updateResponse.status_code >= 400: #400 bad requests
      print("updateACFFields: Failed API call to update ACF Fields: " + updateApiRequestURL);
      print('[!] [{0}] Bad Request, poorly formatted JSON'.format(updateResponse.status_code))
      if loggingLevel == 1:
        print("updateACFFields: returning None API to update ACF fields failed with 400 error");
        print("Failing content was" + data)
      return None
    elif updateResponse.status_code >= 300: #300 redirects
      print("updateACFFields: Failed API call to update ACF Fields: " + updateApiRequestURL);
      print('[!] [{0}] Unexpected redirect.'.format(updateResponse.status_code))
      if loggingLevel == 1:
        print("updateACFFields: returning None API to update ACF fields failed with 400 error");
      return None
    elif updateResponse.status_code == 200: #IF it worked...
      json_response = json.loads(updateResponse.text);
      if loggingLevel == 1:
        print("updateACFFields: returning TRUE as API call to update ACF fields succeeded");
        print("Response: " + str(json_response))
      return True; #finally we hit the full success and return a success
    else:
      print("updateACFFields: Failed API call to update ACF Fields: " + updateApiRequestURL);
      print('[?] Unexpected Error: [HTTP {0}]: Content: {1}'.format(updateResponse.status_code, str(updateResponse.content)))
      if loggingLevel == 1:
        print("updateACFFields: returning None API to update ACF fields failed with unhandled error");
      return None
  
  def existsInWordPress(self):
    """Method that returns True if the given book already exists in wordPress False if it doesnt exist, and None if it failed. Currently uses ISBN# as the key."""
    isbnToCheck = self.isbn;
    if loggingLevel == 1:
        print("existsInWordPress: Defined isbn# to check as: " + str(isbnToCheck));
    checkExistsAPIURL = '{0}/wp-json/wp/v2/book?_fields=id,acf'.format(WPBaseURL);
    if loggingLevel == 1:
      print("Making API call to get ISBN and books in WordPress: " + checkExistsAPIURL)
    checkExistsResponse = requests.get(checkExistsAPIURL);
    if checkExistsResponse.status_code >= 500: #500 errors
      print("existsInWordPress: Failed API call tring to see: " + checkExistsAPIURL);
      print('[!] [{0}] Server Error'.format(checkExistsResponse.status_code))
      if loggingLevel == 1:
        print("existsInWordPress: returning None as API to GET if a book exists in WP by ISBN# failed with 500 error.");
      return None
    elif checkExistsResponse.status_code == 404: #404 not found error
      print("existsInWordPress: Failed API call to: " + checkExistsAPIURL);
      print('[!] [{0}] 404 Not Found: [{1}]'.format(checkExistsResponse.status_code,api_url))
      if loggingLevel == 1:
        print("existsInWordPress: returning None as API to GET if a book exists in WP by ISBN# failed with 404 error.");
      return None
    elif checkExistsResponse.status_code == 401: #401 not auth
      print("existsInWordPress: Failed API call to: " + checkExistsAPIURL);
      print('[!] [{0}] Authentication Failed'.format(checkExistsResponse.status_code))
      if loggingLevel == 1:
        print("existsInWordPress: returning None as API to GET if a book exists in WP by ISBN# failed with 401 error.");
      return None
    elif checkExistsResponse.status_code >= 400: #400 bad requests
      print("existsInWordPress: Failed API call to: " + checkExistsAPIURL);
      print('[!] [{0}] Bad Request, poorly formatted JSON'.format(checkExistsResponse.status_code))
      if loggingLevel == 1:
        print("existsInWordPress: returning None as API to GET if a book exists in WP by ISBN# failed with 400 error.");
        print("The content of the failed response is: " + checkExistsResponse.content)
      return None
    elif checkExistsResponse.status_code >= 300: #300 redirects
      print("existsInWordPress: Failed API call to: " + checkExistsAPIURL);
      print('[!] [{0}] Unexpected redirect.'.format(checkExistsResponse.status_code))
      if loggingLevel == 1:
        print("existsInWordPress: returning None as API to GET if a book exists in WP by ISBN# failed with 300 error.");
      return None
    elif checkExistsResponse.status_code == 200: #IF it worked...
      checkExistsjson_response = json.loads(checkExistsResponse.text);
      if loggingLevel == 1:
        print("existsInWordPress: API to GET if a book exists in WP by ISBN# succeeded. Response was: " + str(checkExistsjson_response));
      #Iterate through the results to see if the ISBN exists for any books that already exist in WordPress
      for i in checkExistsjson_response:
        try:
          currentISBN = str(i['acf']['isbn']);
          if isbnToCheck == currentISBN:
            if loggingLevel == 1:
              print("existsInWordPress: Matching ISBN# found in WordPress already. Returning TRUE");
            return True;
          else:
            if loggingLevel == 1:
              print("existsInWordPress: returning False as no matching ISBN# was found");
            return False;
        except:
          if loggingLevel == 1:
              print("existsInWordPress: Returning NONE. Try/catch error in for loop iterating for ISBN#s");
          return None;
    else:
      print("existsInWordPress: Failed API call to: " + checkExistsAPIURL);
      print('[?] Unexpected Error: [HTTP {0}]: Content: {1}'.format(checkExistsResponse.status_code, checkExistsResponse.content))
      if loggingLevel == 1:
              print("existsInWordPress: Returning NONE. Unknown error when making API call to get list of books and ISBN#s");
      return None